<h1> <center> Example of Earthquake and Tsunami Building Damage with IN-CORE </center> </h1>

This Jupyter notebook contains provides an example of computing building damage using [IN-CORE](https://incore.ncsa.illinois.edu/). This notebook uses the Seaside, Oregon testbed. This notebook is available as an example from the following link: https://github.com/22dylan/pyincore_notebooks/tree/master/20231217_in-core-how-to


<!-- ![seaside](images/seaside.png) -->

<!-- <div>
<img src="attachment:images/seaside.png" width="400">
</div> -->

<!-- In this notebook, the seismic-tsunami hazard posed by the Cascadia Subduction Zone (CSZ) is considered to demonstrate the multi-hazard capabilities in pyIncore. The CSZ is an approximately 1,000 km long fault located between Cape Mendocino California and Vancouver Island, Canada, which separates the Juan de Fuca and North America plates. Rupture of the CSZ will result in a megathrust earthquake and lead to a subsequent tsunami. The city of Seaside is located along the northern Oregon coast, and has been selected as a testbed community for this analysis. -->


<!-- ![flowchart](images/flowchart0.png) -->
<center><img src="images/flowchart0.png" width="80%"/></center>



## <center> Import IN-CORE and Necessary Modules </center>
This section imports all necessary python packages and pyincore modules. This section also establishes a connection to the IN-CORE server by initializing an IN-CORE client. 

<center><img src="images/flowchart1.png" width="50%"/></center>


In [ ]:
import os
import pandas as pd

from pyincore import IncoreClient, Dataset, DataService, FragilityService, MappingSet
from pyincore.analyses.buildingdamage import BuildingDamage
from pyincore.analyses.cumulativebuildingdamage import CumulativeBuildingDamage

client = IncoreClient()
data_service = DataService(client)
fragility_service = FragilityService(client)

In [ ]:
output_path = os.path.join(os.getcwd(), 'output')
if not os.path.exists(output_path):
    os.makedirs(output_path)

## <center> Initial Community Description </center> 

This section defines the initial community description before the building damage code is ran. This section corresponds to the following part of the flowchart.

<center><img src="images/flowchart2.png" width="50%"/></center>

Loading the building inventory from IN-CORE. The building inventory is saved on IN-CORE's dataservers available [here](https://incore.ncsa.illinois.edu/DataViewer). Just paste the dataset id ```613ba5ef5d3b1d6461e8c415``` into the search bar.

<!-- <center><img src="images/seaside.png" width="70%"/></center> -->

In [ ]:
bldg_dataset_id = "613ba5ef5d3b1d6461e8c415"        # defining building dataset (GIS point layer)
bldg_dataset = Dataset.from_data_service(bldg_dataset_id, data_service)
gdf = bldg_dataset.get_dataframe_from_shapefile()
gdf.set_index("guid", inplace=True)


Plotting the building inventory

In [ ]:
gdf.explore(
        column="struct_typ", # column for archetype info
        tooltip=["struct_typ", "rmv_improv"], 
        popup=True,  
        tiles="CartoDB positron"
)

## <center> Building Damage </center>
This section uses IN-CORE to compute building damge from seismic and tsunami hazards. This section corresponds to the following part of the flowchart.

<center><img src="images/flowchart3.png" width="50%"/></center>

Define recurrence interval

In [ ]:
recurrence_interval = int(input("Reccurrence interval to consider: "))
eq_hazard_dict = {
            100: "5dfa4058b9219c934b64d495", 
            250: "5dfa41aab9219c934b64d4b2",
            500: "5dfa4300b9219c934b64d4d0",
            1000: "5dfa3e36b9219c934b64c231",
            2500: "5dfa4417b9219c934b64d4d3", 
            5000: "5dfbca0cb9219c101fd8a58d",
            10000: "5dfa51bfb9219c934b68e6c2"
            }
ts_hazard_dict = {
            100: "5bc9e25ef7b08533c7e610dc", 
            250: "5df910abb9219cd00cf5f0a5",
            500: "5df90e07b9219cd00ce971e7",
            1000: "5df90137b9219cd00cb774ec",
            2500: "5df90761b9219cd00ccff258",
            5000: "5df90871b9219cd00ccff273",
            10000: "5d27b986b9219c3c55ad37d0"
            }
eq_hazard_id = eq_hazard_dict[recurrence_interval]
ts_hazard_id = ts_hazard_dict[recurrence_interval]

#### Earthquake Damage

In [ ]:
bldg_dmg = BuildingDamage(client)   

# defining building dataset (GIS point layer)
bldg_dmg.set_input_dataset("buildings", bldg_dataset)

# setup fragility mappings
mapping_id = "5d2789dbb9219c3c553c7977"
eq_mapping_set = MappingSet(fragility_service.get_mapping(mapping_id))
bldg_dmg.set_input_dataset('dfr3_mapping_set', eq_mapping_set)

# set parameters
result_name = os.path.join(output_path, 'Bldg-EQ-{}yr' .format(recurrence_interval))
bldg_dmg.set_parameter("hazard_type", "earthquake")
bldg_dmg.set_parameter("num_cpu", 4)
bldg_dmg.set_parameter("hazard_id", eq_hazard_id)
bldg_dmg.set_parameter("result_name", result_name)

# run analysis
bldg_dmg.run_analysis()
print("earthquake building damage done.")

#### Tsunami Damage

In [ ]:
bldg_dmg = BuildingDamage(client)   

# defining building dataset (GIS point layer)
bldg_dmg.set_input_dataset("buildings", bldg_dataset)

# setup fragility mappings
mapping_id = "5d279bb9b9219c3c553c7fba"
ts_mapping_set = MappingSet(fragility_service.get_mapping(mapping_id))
bldg_dmg.set_input_dataset('dfr3_mapping_set', ts_mapping_set)

# set parameters
result_name = os.path.join(output_path, 'Bldg-TS-{}yr' .format(recurrence_interval))
bldg_dmg.set_parameter("hazard_type", "tsunami")
bldg_dmg.set_parameter("num_cpu", 4)
bldg_dmg.set_parameter("hazard_id", ts_hazard_id)
bldg_dmg.set_parameter("result_name", result_name)

# run analysis
bldg_dmg.run_analysis()
print("tsunami building damage done.")

#### Cumulative Damage

In [ ]:
# initializing pyIncore cumulative building damage
cumulative_bldg_dmg = CumulativeBuildingDamage(client)
cumulative_bldg_dmg.set_parameter("num_cpu", 4)

# reading in damage results from above analysis
eq_damage_results_csv = os.path.join(output_path, 'Bldg-EQ-{}yr.csv' .format(recurrence_interval))
ts_damage_results_csv = os.path.join(output_path, 'Bldg-TS-{}yr.csv' .format(recurrence_interval))

# loading datasets from CSV files into pyincore
eq_damage_dataset = Dataset.from_file(eq_damage_results_csv, "ergo:buildingDamageVer5")
tsu_damage_dataset = Dataset.from_file(ts_damage_results_csv, "ergo:buildingDamageVer5")

cumulative_bldg_dmg.set_input_dataset("eq_bldg_dmg", eq_damage_dataset)
cumulative_bldg_dmg.set_input_dataset("tsunami_bldg_dmg", tsu_damage_dataset)

# defining path to output 
result_name = os.path.join(output_path, 'Bldg-CM-{}yr' .format(recurrence_interval))
cumulative_bldg_dmg.set_parameter("result_name", result_name)

# running analysis
cumulative_bldg_dmg.run_analysis()
print("cumulative building damage done.")

#### Visualize Building Damage Results

In [ ]:
# cumulative_bldg_dmg
ds_result = cumulative_bldg_dmg.get_output_dataset("combined-result").get_dataframe_from_csv()
ds_result.set_index("guid", inplace=True)
gdf = pd.merge(gdf, ds_result, left_index=True, right_index=True)
gdf['E_DS'] = gdf['DS_0']*0 + gdf['DS_1']*1 + gdf['DS_2']*2 + gdf['DS_3']*3

In [ ]:
gdf.explore(
        column="E_DS", # column for archetype info
        tooltip=["E_DS", "DS_0", "DS_1", "DS_2", "DS_3"],
        vmin=0,
        vmax=3,
        popup=True,  
        cmap="RdYlGn_r",  # use "Set1" matplotlib colormap),
        tiles="CartoDB positron"
)

## <center> Economic Losses </center>
Direct economic losses can be computed in IN-CORE by chaining the output from the building damage analysis to the mean damage and economic loss anlayses. The methods for computing economic losses are from [Bai et al. (2009)](https://doi.org/10.1061/(ASCE)0733-9445(2009)135:10(1155)). An example of using IN-CORE to compute economic losses in Seaside is available in Figure 9 of [this paper](https://doi.org/10.1016/j.ijdrr.2023.103755). The mean damage is first computed by multiplying the probability of being in each damage state with damage ratios and taking the sum. The economic losses are then computed by multiplying the mean damage by each building's value.

This can be done in IN-CORE by chaining output from the building damage anlaysis to the following analyses:
+ [Mean damage](https://incore.ncsa.illinois.edu/doc/incore/analyses/mean_dmg.html)
+ [Building Economic Loss](https://incore.ncsa.illinois.edu/doc/incore/analyses/building_loss.html)

Remember that outputs from one analysis can be obtained and chained to another using the following piece of code: \
```bldg_dmg.get_output_dataset("ds_result")```

*Note that as of now, the cumulative building damage output cannot be directly chained to the mean damage and economic loss analysis in IN-CORE. 

#### Mean Damage
The mean damage analysis needs a damage ratio id. The following can be used: ```5a284f2ec7d30d13bc08209a```. This can be viewed on the [IN-CORE data-viewer](https://incore.ncsa.illinois.edu/DataViewer)

In [ ]:
from pyincore.analyses.meandamage import MeanDamage


#### Economic Losses

In [ ]:
from pyincore.analyses.buildingeconloss import BuildingEconLoss
